In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import random

spark = SparkSession.builder \
    .appName("StreamPulse-PlanAudit") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()


In [ ]:
random.seed(42)

# Listening events (large table - 500K rows)
events_data = []
for i in range(500000):
    events_data.append((
        f"EVT-{i+1:07d}",
        f"USR-{random.randint(1, 100000):06d}",
        f"TRK-{random.randint(1, 50000):06d}",
        f"ART-{random.randint(1, 5000):05d}",
        random.randint(10, 300),
        random.choice([True, False]),
        random.choice(["mobile", "desktop", "smart_speaker", "tablet"]),
        random.choice(["free", "premium"]),
        f"202{random.randint(3,4)}-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
    ))

events = spark.createDataFrame(events_data,
    ["event_id", "user_id", "track_id", "artist_id", "duration_sec",
     "completed", "device", "tier", "event_date"]) \
    .withColumn("event_date", col("event_date").cast("date")) \
    .withColumn("year", year(col("event_date"))) \
    .withColumn("month", month(col("event_date")))

events.write.parquet("audit_data/events", mode="overwrite", partitionBy=["year"])

# Artists (small table - 5K rows)
artist_data = [(f"ART-{i+1:05d}", f"Artist {i+1}",
                random.choice(["Pop", "Rock", "Hip-Hop", "Jazz", "Electronic"]),
                random.choice(["US", "UK", "KR", "JP", "DE"]))
               for i in range(5000)]
artists = spark.createDataFrame(artist_data, ["artist_id", "name", "genre", "country"])
artists.write.parquet("audit_data/artists", mode="overwrite")

# Tracks (medium table - 50K rows)
track_data = [(f"TRK-{i+1:06d}", f"Track {i+1}",
               f"ART-{random.randint(1, 5000):05d}",
               random.randint(60, 400),
               random.randint(2018, 2024))
              for i in range(50000)]
tracks = spark.createDataFrame(track_data,
    ["track_id", "title", "artist_id", "track_duration", "release_year"])
tracks.write.parquet("audit_data/tracks", mode="overwrite")

# Reload from Parquet
events = spark.read.parquet("audit_data/events")
artists = spark.read.parquet("audit_data/artists")
tracks = spark.read.parquet("audit_data/tracks")

print(f"Events: {events.count()} | Artists: {artists.count()} | Tracks: {tracks.count()}")


In [ ]:
# Query 1: Simple filter and select
q1 = events.filter(col("year") == 2024) \
    .filter(col("completed") == True) \
    .select("event_id", "user_id", "duration_sec")

print("QUERY 1: Simple filter and select")
q1.explain(mode="formatted")


### Interpreting the formatted explain output

Assuming a typical Spark setup (Parquet/Delta source, year as a partition column), here’s how to read it:


| Aspect                 | Your Finding                                                                                      |
| ---------------------- | ------------------------------------------------------------------------------------------------- |
| **Scan type**          | `FileScan` (typically Parquet or Delta)                                                           |
| **PartitionFilters**   | `year = 2024`                                                                                     |
| **PushedFilters**      | `completed = true`                                                                                |
| **ReadSchema columns** | `event_id`, `user_id`, `duration_sec` *(plus any required metadata columns)*                      |
| **Exchange count**     | `0`                                                                                               |
| **Assessment**         | Very efficient scan: partition pruning + predicate pushdown + column pruning; no shuffle required |


In [ ]:
# Query 2: Join events with artists

q2 = events.join(artists, "artist_id") \
    .filter(col("year") == 2024) \
    .select("event_id", "name", "genre", "duration_sec")

print("QUERY 2: Events JOIN Artists (filter after join)")
q2.explain(mode="formatted")



| Aspect                 | Your Finding                                                                                                                 |
| ---------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| **Join strategy**      | `BroadcastHashJoin`                                                                                                          |
| **Artists table size** | ~5K rows (small!)                                                                                                            |
| **Exchange count**     | `0`                                                                                                                          |
| **Could broadcast?**   | Yes (artists is well below broadcast threshold)                                                                              |
| **Filter placement**   | Applied **after join** (not pushed below join)                                                                               |
| **Assessment**         | Join is efficient due to broadcast, but filter placement is suboptimal; filtering `events` before the join would reduce work |


In [ ]:
# Query 3: Three-table join
q3 = events.join(tracks, "track_id") \
    .join(artists, "artist_id") \
    .filter(col("year") == 2024) \
    .filter(col("genre") == "Pop") \
    .groupBy("name") \
    .agg(count("*").alias("play_count"), avg("duration_sec").alias("avg_duration"))

print("QUERY 3: Three-table join with aggregation")
q3.explain(mode="formatted")


| Aspect                   | Your Finding                                                                                                         |
| ------------------------ | -------------------------------------------------------------------------------------------------------------------- |
| **Join 1 strategy**      | `SortMergeJoin` (events ⨝ tracks)                                                                                    |
| **Join 2 strategy**      | `BroadcastHashJoin` (artists broadcast)                                                                              |
| **Total Exchange count** | `2`                                                                                                                  |
| **Filter on year?**      | Yes — **partition filter**, but applied *after joins*                                                                |
| **Filter on genre?**     | Yes — **pushed to artists scan**                                                                                     |
| **Assessment**           | Correct but expensive plan: shuffle-heavy first join, broadcast second join; filters applied late increase join cost |


In [ ]:
# Query 4: Aggregation with multiple actions (simulated)
enriched = events.join(artists, "artist_id").filter(col("year") == 2024)

print("QUERY 4a: Genre aggregation")
q4a = enriched.groupBy("genre").agg(count("*").alias("plays"))
q4a.explain(mode="formatted")

print("\nQUERY 4b: Device aggregation (same enriched source)")
q4b = enriched.groupBy("device").agg(avg("duration_sec").alias("avg_dur"))
q4b.explain(mode="formatted")


| Aspect                                | Your Finding                                                                                |
| ------------------------------------- | ------------------------------------------------------------------------------------------- |
| **Does 4a and 4b share computation?** | ❌ No                                                                                        |
| **Is enriched cached?**               | ❌ No                                                                                        |
| **Redundant work**                    | Join + filter are executed **twice**                                                        |
| **Assessment**                        | Inefficient: repeated expensive join and scan; caching `enriched` would avoid recomputation |


In [ ]:
# Query 5: Self-join pattern
popular = events.groupBy("track_id").agg(count("*").alias("play_count")) \
    .filter(col("play_count") > 10)

q5 = events.join(popular, "track_id") \
    .select("event_id", "user_id", "track_id", "play_count")

print("QUERY 5: Self-reference (events aggregated then joined back)")
q5.explain(mode="formatted")


| Aspect                                | Your Finding                                                                                               |
| ------------------------------------- | ---------------------------------------------------------------------------------------------------------- |
| **How many times is events scanned?** | **2 times**                                                                                                |
| **Exchange count**                    | **2**                                                                                                      |
| **Join strategy**                     | `SortMergeJoin`                                                                                            |
| **Could caching help?**               | ✅ Yes                                                                                                      |
| **Assessment**                        | Correct logic but expensive; double scan + shuffle; caching or restructuring can significantly reduce cost |


# StreamPulse Execution Plan Audit Report

## Summary
- **Total queries analyzed:** 5
- **Queries needing optimization:** 4 (Queries 2–5)
- **Most common issue:** Filters applied *after* joins and repeated recomputation due to missing caching
- **Estimated total improvement potential:** 30–60% reduction in shuffle I/O and CPU for complex queries

---

## Query-by-Query Findings

### Query 1: Simple Filter
- **Status:** Efficient
- **Issues found:** None
- **Recommendation:**  
  No changes needed; already benefits from partition pruning, predicate pushdown, and column pruning.

---

### Query 2: Events–Artists Join
- **Status:** Needs Work
- **Issues found:**
  - Filter on `year` applied after join
  - Unnecessary rows participate in the join
- **Recommendation:**
  - Filter `events` on `year = 2024` *before* joining
  - Retain broadcast join for `artists`

---

### Query 3: Three-Table Join with Aggregation
- **Status:** Needs Work
- **Issues found:**
  - Expensive shuffle-based join between `events` and `tracks`
  - Partition filter (`year`) applied too late
  - Large intermediate datasets before aggregation
- **Recommendation:**
  - Push filters (`year`, `genre`) to base tables before joins
  - Reduce join input size prior to shuffle
  - Preserve broadcast join for `artists`

---

### Query 4: Aggregation with Multiple Actions
- **Status:** Needs Work
- **Issues found:**
  - Same join and filter recomputed for multiple actions
  - No shared computation between Query 4a and 4b
- **Recommendation:**
  - Cache or persist the shared `enriched` DataFrame
  - Materialize cache before running multiple aggregations

---

### Query 5: Self-Join Pattern
- **Status:** Needs Work
- **Issues found:**
  - `events` scanned twice
  - Multiple shuffles (aggregation + join)
  - SortMergeJoin used even when aggregated side may be small
- **Recommendation:**
  - Cache the aggregated `popular` DataFrame
  - Broadcast `popular` if size permits
  - Consider rewriting using window functions to avoid self-join

---

## Priority Recommendations
1. **Push filters as early as possible** (especially partition filters before joins)
2. **Cache shared intermediate results** used by multiple actions
3. **Leverage broadcast joins explicitly** for small dimension or aggregated tables

---

## Configuration Recommendations
- **spark.sql.autoBroadcastJoinThreshold:**  
  Keep default (~10MB) or increase slightly (e.g., 20–50MB) if dimension tables are consistently small

- **spark.sql.shuffle.partitions:**  
  Tune based on cluster size (e.g., 100–200 instead of default 200 for mid-sized workloads)

- **Caching strategy:**  
  Cache only reused, expensive intermediates (joins, filtered fact tables)  
  Prefer `MEMORY_AND_DISK` for large cached DataFrames

In [ ]:
from pyspark.sql.functions import col, count, avg, broadcast

# -------------------------------
# Query 1: Already efficient
# -------------------------------
q1_optimized = events \
    .filter(col("year") == 2024) \
    .filter(col("completed") == True) \
    .select("event_id", "user_id", "duration_sec")
# No changes needed; partition pruning and column pruning already applied

# -------------------------------
# Query 2: Push filter before join and broadcast artists
# -------------------------------
q2_optimized = events \
    .filter(col("year") == 2024) \
    .join(broadcast(artists), "artist_id") \
    .select("event_id", "name", "genre", "duration_sec")

# -------------------------------
# Query 3: Push filters early, preserve broadcast for small artists
# -------------------------------
events_filtered = events.filter(col("year") == 2024)
artists_filtered = artists.filter(col("genre") == "Pop")

q3_optimized = events_filtered \
    .join(tracks, "track_id") \
    .join(broadcast(artists_filtered), "artist_id") \
    .groupBy("name") \
    .agg(
        count("*").alias("play_count"),
        avg("duration_sec").alias("avg_duration")
    )

# -------------------------------
# Query 4: Cache enriched DataFrame for multiple aggregations
# -------------------------------
enriched = events \
    .join(broadcast(artists), "artist_id") \
    .filter(col("year") == 2024) \
    .cache()

# Aggregation by genre
q4a_optimized = enriched.groupBy("genre") \
    .agg(count("*").alias("plays"))

# Aggregation by device
q4b_optimized = enriched.groupBy("device") \
    .agg(avg("duration_sec").alias("avg_dur"))

# -------------------------------
# Query 5: Cache aggregated popular tracks and broadcast if small
# -------------------------------
popular = events.groupBy("track_id") \
    .agg(count("*").alias("play_count")) \
    .filter(col("play_count") > 10) \
    .cache()  # cache for reuse

q5_optimized = events \
    .join(broadcast(popular), "track_id") \
    .select("event_id", "user_id", "track_id", "play_count")

✅ Key optimization patterns applied

- Filter pushdown: Always filter events or artists early to reduce shuffle and join size

- Broadcast joins: Used for small tables (artists) or aggregated tables (popular)

- Caching shared intermediate results: enriched (Query 4) and popular (Query 5)

- Column pruning: Only select needed columns after joins or filters

- Avoid multiple scans: By caching, repeated actions reuse already-computed DataFrames